<a href="https://colab.research.google.com/github/usneek/Multilingual-Conversational-Question-Answering/blob/main/Q_A_with_context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install simpletransformers

     |████████████████████████████████| 248 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
     |████████████████████████████████| 6.8 MB 32.4 MB/s 
     |████████████████████████████████| 1.7 MB 38.1 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 9.7 MB 38.2 MB/s 
     |████████████████████████████████| 3.5 MB 44.0 MB/s 
     |████████████████████████████████| 312 kB 55.5 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 
     |████████████████████████████████| 895 kB 50.6 MB/s 
     |████████████████████████████████| 67 kB 3.5 MB/s 
     |████████████████████████████████| 143 kB 50.4 MB/s 
     |████████████████████████████████| 180 kB 50.2 MB/s 
     |████████████████████████████████| 97 kB 5.6 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 133 kB 47.1 MB/s 
     |████████████████████████████████| 1.1 MB 57.4 MB/s 
     |█████████████████

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(0)
from keras.regularizers import l2
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import f1_score
np.random.seed(1)
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import re
import string
import seaborn as sns
tf.get_logger().setLevel('ERROR')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
data_file = open('drive/My Drive/Conversational Question Answering System Code Notebooks/data.json',encoding='utf-8').read()
intents = json.loads(data_file)

In [ ]:
sentence=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        sentence.append(pattern)

In [ ]:
classes=[]
for intent in intents['intents']:
  for pattern in intent['patterns']:
    classes.append(intent['tag'])

In [ ]:
print(sentence)

['सीज़ेंट्रिक्स कॉलबैक', 'सीजेड कैक्लबैक', 'सीज़ेंट्रिक्स कॉल बैक', 'C केंद्र कॉल बैक', 'सीज़ेंट्रिक्स कॉल बैक', 'वापस कॉल करें', 'क्या आप कॉल बैक सुविधा प्रदान करते हैं', 'कॉल बैक सॉल्यूशन', 'एक कॉल बैक समाधान की तलाश में', 'कॉल बैक सेवा की आवश्यकता है', 'कॉल बैक सॉल्यूशन है', 'सीज़ेंट्रिक्स कॉलबैक क्या है', 'क्या आप मुझे सीज़ेंट्रिक्स कॉलबैक समाधान में मदद कर सकते हैं', 'क्या आप मुझे सीज़ेंट्रिक्स कॉल बैक सॉल्यूशन में मदद कर सकते हैं', 'क्या आप मुझे कॉल बैक समाधान के बारे में बता सकते हैं', 'ठीक है धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताओ', 'धन्यवाद, मुझे कॉलबैक के बारे में बताएं', 'थैंक्यू, कॉल बैक क्या है', 'अच्छा, क्या आप मुझे कॉलबैक के बारे में बता सकते हैं?', 'धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताएं', ' सीज़ेंट्रिक्स हेल्पडेस्क', 'सीज़ेंट्रिक्सहेल्पडेस्क', 'सीज़ेंट्रिक्स मदद', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'मदद', 'सहायता केंद्र', 'हेल्प डेस्क', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'क्या आप हेल्पडेस्क सुविधा प्रदान करते हैं', 'एक हेल्प डेस्क समाधान की तलाश में', 'मदद ड

In [ ]:
print(classes)

['cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'no

In [ ]:
from numpy import array
from numpy import argmax
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
encoder = LabelEncoder()
encoder.fit(classes)
encoded_Y = encoder.transform(classes)
encoded_Y=np_utils.to_categorical(encoded_Y)

In [ ]:
print(encoded_Y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
X=[]
for i in sentence:
  X.append(i)

In [ ]:
print(X)

['सीज़ेंट्रिक्स कॉलबैक', 'सीजेड कैक्लबैक', 'सीज़ेंट्रिक्स कॉल बैक', 'C केंद्र कॉल बैक', 'सीज़ेंट्रिक्स कॉल बैक', 'वापस कॉल करें', 'क्या आप कॉल बैक सुविधा प्रदान करते हैं', 'कॉल बैक सॉल्यूशन', 'एक कॉल बैक समाधान की तलाश में', 'कॉल बैक सेवा की आवश्यकता है', 'कॉल बैक सॉल्यूशन है', 'सीज़ेंट्रिक्स कॉलबैक क्या है', 'क्या आप मुझे सीज़ेंट्रिक्स कॉलबैक समाधान में मदद कर सकते हैं', 'क्या आप मुझे सीज़ेंट्रिक्स कॉल बैक सॉल्यूशन में मदद कर सकते हैं', 'क्या आप मुझे कॉल बैक समाधान के बारे में बता सकते हैं', 'ठीक है धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताओ', 'धन्यवाद, मुझे कॉलबैक के बारे में बताएं', 'थैंक्यू, कॉल बैक क्या है', 'अच्छा, क्या आप मुझे कॉलबैक के बारे में बता सकते हैं?', 'धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताएं', ' सीज़ेंट्रिक्स हेल्पडेस्क', 'सीज़ेंट्रिक्सहेल्पडेस्क', 'सीज़ेंट्रिक्स मदद', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'मदद', 'सहायता केंद्र', 'हेल्प डेस्क', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'क्या आप हेल्पडेस्क सुविधा प्रदान करते हैं', 'एक हेल्प डेस्क समाधान की तलाश में', 'मदद ड

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 3.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=8a7809082794ed140fb408aeb05bdea1d876cd0d0a7e4bff9d865a04ddae1989
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('google/muril-base-cased')

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/google_muril-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentence_embedding=sbert_model.encode(X)

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(sentence_embedding.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(33, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
hist = model.fit(sentence_embedding,encoded_Y, epochs=300, batch_size=10, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/300
53/53 [==============================] - 1s 4ms/step - loss: 3.4876 - accuracy: 0.0453
Epoch 2/300
53/53 [==============================] - 0s 4ms/step - loss: 3.4722 - accuracy: 0.0377
Epoch 3/300
53/53 [==============================] - 0s 4ms/step - loss: 3.4592 - accuracy: 0.0340
Epoch 4/300
53/53 [==============================] - 0s 5ms/step - loss: 3.4493 - accuracy: 0.0453
Epoch 5/300
53/53 [==============================] - 0s 6ms/step - loss: 3.4379 - accuracy: 0.0604
Epoch 6/300
53/53 [==============================] - 0s 7ms/step - loss: 3.4256 - accuracy: 0.0547
Epoch 7/300
53/53 [==============================] - 0s 6ms/step - loss: 3.4133 - accuracy: 0.0642
Epoch 8/300
53/53 [==============================] - 0s 7ms/step - loss: 3.4035 - accuracy: 0.0604
Epoch 9/300
53/53 [==============================] - 0s 8ms/step - loss: 3.3992 - accuracy: 0.0906
Epoch 10/300
53/53 [==============================] - 0s 8ms/step - loss: 3.3601 - accuracy: 0.0925
Epoch 11/

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel
model2 = QuestionAnsweringModel('bert', 'salti/bert-base-multilingual-cased-finetuned-squad', use_cuda=False)

Downloading:   0%|          | 0.00/822 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/676M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/264 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
model3 = QuestionAnsweringModel('bert', 'google/muril-base-cased', use_cuda=False)

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/909M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the m

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [ ]:
import random
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
final=['','affirm','crm_application','cz_acd','cz_bar','cz_bot','cz_callback',
 'cz_callblast','cz_chat','cz_conference','cz_dialer','cz_guide',
 'cz_helpdesk','cz_ivr','cz_lms','cz_magic_call','cz_missed_call',
 'cz_mobile','cz_screen_logger','cz_voice_logger','deny','greet','home',
 'manual_dialer','non_voice_process','predictive_dialer','preview_dialer',
 'product','progressive_dialer','self_service_solutions','thankyou',
 'voice_process','work_force_management']

In [ ]:
def chatbot_response(text):
  text=[text]
  em=sbert_model.encode(text)
  res = model.predict(em)[0]
  ERROR_THRESHOLD = 0.3
  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
      return_list.append({"intent": final[r[0]], "probability": str(r[1])})
  context = getResponse(return_list, intents)
  to_predict = [{'context': context,'qas': [{'question':text, 'id': '2'}]}]
  res=model3.predict(to_predict)
  print(res)
 

"सीज़ेंट्रिक्स डायलर न केवल आपको डेटाबेस आयात करने का एक आसान विकल्प देता है, बल्कि वेबसाइट, सीआरएम, एलएमएस आदि जैसे तीसरे पक्ष के लीड स्रोतों के साथ आसानी से एकीकृत करता है ताकि लीड सीधे डायलर से वास्तविक समय में जुड़ सकें। चार प्रकार के डायलर हैं 1. प्रडिक्टिव डायलर 2. प्रॊगरेशिव डायलर 3. प्रीव्यू डायलर 4. मैनुअल डायलर"

In [ ]:
chatbot_response('c-zentrix डायलर कितने प्रकार के होते हैं')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6689.48it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '2', 'answer': ['3. प्रीव्यू डायलर 4. मैनुअल डायल', '3.', '3. प्रीव्यू डायलर 4.', '3. प्रीव्यू डायल', 'न केवल आपको डेटाबेस आयात करने का एक आसान विकल्प देता है, बल्कि वेबसाइट, सीआरएम, एलएमएस आदि जैसे तीसरे पक्ष के लीड स्रोतों के साथ आसानी से एकीकृत करता है ताकि लीड सीधे डायलर से वास्तविक समय में जुड़ सकें। चार प्रकार के डायलर हैं 1. प्रडिक्टिव डायलर 2. प्रॊ', 'न केवल आपको डेटाबेस आयात करने का एक आसान विकल्प देता है, बल्कि वेबसाइट, सीआरएम, एलएमएस आदि जैसे तीसरे पक्ष के लीड स्रोतों के साथ आसानी से एकीकृत करता है ताकि लीड सीधे डायलर से वास्तविक समय में जुड़ सकें। चार प्रकार के डायल', 'आदि जैसे तीसरे पक्ष के लीड स्रोतों के साथ आसानी से एकीकृत करता है ताकि लीड सीधे डायलर से वास्तविक समय में जुड़ सकें। चार प्रकार के डायलर हैं 1. प्रडिक्टिव डायलर 2. प्रॊ', 'आदि जैसे तीसरे पक्ष के लीड स्रोतों के साथ आसानी से एकीकृत करता है ताकि लीड सीधे डायलर से वास्तविक समय में जुड़ सकें। चार प्रकार के डायल', '3. प्रीव्यू डायलर 4. मैनुअल डायलर', 'न केवल आपको डेटाबेस आयात करने का एक आसान विकल्प देता है

In [ ]:
chatbot_response("सी-ज़ेंट्रिक्स डायलर क्या है")

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6721.64it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '2', 'answer': ['3. प्रीव्यू डायलर 4. मैनुअल डायल', '3.', '3. प्रीव्यू डायलर 4.', '3. प्रीव्यू डायल', 'न केवल आपको डेटाबेस आयात करने का एक आसान विकल्प देता है, बल्कि वेबसाइट, सीआरएम, एलएमएस आदि जैसे तीसरे पक्ष के लीड स्रोतों के साथ आसानी से एकीकृत करता है ताकि लीड सीधे डायलर से वास्तविक समय में जुड़ सकें। चार प्रकार के डायलर हैं 1. प्रडिक्टिव डायलर 2. प्रॊ', 'न केवल आपको डेटाबेस आयात करने का एक आसान विकल्प देता है, बल्कि वेबसाइट, सीआरएम, एलएमएस आदि जैसे तीसरे पक्ष के लीड स्रोतों के साथ आसानी से एकीकृत करता है ताकि लीड सीधे डायलर से वास्तविक समय में जुड़ सकें। चार प्रकार के डायल', 'आदि जैसे तीसरे पक्ष के लीड स्रोतों के साथ आसानी से एकीकृत करता है ताकि लीड सीधे डायलर से वास्तविक समय में जुड़ सकें। चार प्रकार के डायलर हैं 1. प्रडिक्टिव डायलर 2. प्रॊ', 'आदि जैसे तीसरे पक्ष के लीड स्रोतों के साथ आसानी से एकीकृत करता है ताकि लीड सीधे डायलर से वास्तविक समय में जुड़ सकें। चार प्रकार के डायल', '3. प्रीव्यू डायलर 4. मैनुअल डायलर', 'न केवल आपको डेटाबेस आयात करने का एक आसान विकल्प देता है

"सीज़ेंट्रिक्स कॉलबैक समाधान आपके भावी ग्राहकों को लंबे समय तक कॉल पर प्रतीक्षा नहींकरने में मदद करता है। उन्हें तुरंत कॉल करें या उनकी सुविधानुसार कॉल शेड्यूल  करें। आगंतुक या ग्राहक अपना संपर्क नंबर प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर जाएं"

In [ ]:
chatbot_response("क्या आप मुझे कॉल बैक समाधान के बारे में बता सकते हैं")

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6141.00it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '2', 'answer': ['प्रदान', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर जाएं', 'प्रदान कर सकते हैं और', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे।', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है।', 'प्रदान कर', 'नुसार कॉल शेड्यूल करें। आगंतुक या ग्राहक अपना संपर्क नंबर प्रदान', 'नुसार कॉल शेड्यूल', 'नुसार कॉल शेड्यूल करें। आगंतुक या ग्राहक अपना संपर्क नंबर प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर जाएं', 'ज़ेंट्रिक्स कॉलबैक समाधान आपके भावी ग्राहकों को लंबे समय तक कॉल पर प्रतीक्षा नहींकरने में मदद करता है। उन्हें तुरंत कॉल', 'ज़ेंट

In [ ]:
chatbot_response("सीज़ेंट्रिक्स कॉलबैक क्या है")

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6326.25it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '2', 'answer': ['प्रदान', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर जाएं', 'प्रदान कर सकते हैं और', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे।', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है।', 'प्रदान कर', 'नुसार कॉल शेड्यूल करें। आगंतुक या ग्राहक अपना संपर्क नंबर प्रदान', 'नुसार कॉल शेड्यूल', 'नुसार कॉल शेड्यूल करें। आगंतुक या ग्राहक अपना संपर्क नंबर प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर जाएं', 'ज़ेंट्रिक्स कॉलबैक समाधान आपके भावी ग्राहकों को लंबे समय तक कॉल पर प्रतीक्षा नहींकरने में मदद करता है। उन्हें तुरंत कॉल', 'ज़ेंट

In [ ]:
chatbot_response("क्या कॉलबैक ड्रॉपआउट दरों को बचाता है?")

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6442.86it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '2', 'answer': ["समीक्षा करने में सक्षम बनाता है। अधिक जानकारी के लिए लिंक पर जाएं,<a href='https://www.czentrix.com/products/czdialer'>यहाँ क्लिक करें</", "समीक्षा करने में सक्षम बनाता है। अधिक जानकारी के लिए लिंक पर जाएं,<a href='https://", "समीक्षा करने में सक्षम बनाता है। अधिक जानकारी के लिए लिंक पर जाएं,<a href='https://www.czentrix.com/products/", "समीक्षा करने में सक्षम बनाता है। अधिक जानकारी के लिए लिंक पर जाएं,<a href='https://www.czentrix.com/products/czdialer", "x.com/products/czdialer'>यहाँ क्लिक करें</", "समीक्षा करने में सक्षम बनाता है। अधिक जानकारी के लिए लिंक पर जाएं,<a href='https://www.czentrix.com/", "समीक्षा करने में सक्षम बनाता है। अधिक जानकारी के लिए लिंक पर जाएं,<a href='https://www.czentrix.com/products/czdialer'>यहाँ क्लिक", "zentrix.com/products/czdialer'>यहाँ क्लिक करें</", "समीक्षा करने में सक्षम बनाता है। अधिक जानकारी के लिए लिंक पर जाएं,<a href='https:/", "/products/czdialer'>यहाँ क्लिक करें</", "एजेंटों को कॉल करने से पहले संपर्क विवरण, विशिष्ट ग

In [ ]:
chatbot_response(" कॉलबैक के लिए लिंक क्या है")

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 10180.35it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '2', 'answer': ['प्रदान', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर जाएं', 'प्रदान कर सकते हैं और', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे।', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर', 'प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है।', 'प्रदान कर', 'नुसार कॉल शेड्यूल करें। आगंतुक या ग्राहक अपना संपर्क नंबर प्रदान', 'नुसार कॉल शेड्यूल', 'नुसार कॉल शेड्यूल करें। आगंतुक या ग्राहक अपना संपर्क नंबर प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर जाएं', 'ज़ेंट्रिक्स कॉलबैक समाधान आपके भावी ग्राहकों को लंबे समय तक कॉल पर प्रतीक्षा नहींकरने में मदद करता है। उन्हें तुरंत कॉल', 'ज़ेंट